In [40]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [41]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [42]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg' , shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x , (120,120)))
train_images = train_images.map(lambda x: x/255)

In [43]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg' , shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x , (120,120)))
test_images = test_images.map(lambda x: x/255)

In [44]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg' , shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x , (120,120)))
val_images = val_images.map(lambda x: x/255)

##### Prepare label

In [45]:
def load_labels(label_path):
    with open(label_path.numpy() , 'r' , encoding='utf-8') as f:
        label = json.load(f)
        
    return [label['class']] , label['bbox']

In [46]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json' , shuffle=False)
train_labels = train_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

In [47]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json' , shuffle=False)
test_labels = test_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

In [48]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json' , shuffle=False)
val_labels = val_labels.map(lambda x:tf.py_function(load_labels , [x] , [tf.uint8 , tf.float16]))

#### Create final dataset


In [49]:
train = tf.data.Dataset.zip((train_images , train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [50]:
test = tf.data.Dataset.zip((train_images , train_labels))
test = test.shuffle(5000)
test = test.batch(8)
test = test.prefetch(4)

In [51]:
val = tf.data.Dataset.zip((train_images , train_labels))
val = val.shuffle(5000)
val = val.batch(8)
val = val.prefetch(4)

# Strat with model training

In [52]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [55]:
vgg = VGG16(include_top=False)

58889256/58889256 [==============================] - 5s 0us/step


In [63]:
def build_model():
    input_layer = Input(shape=(120 , 120 , 3))
    
    vgg = VGG16(include_top=False)(input_layer)
    
    # Classification model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048 , activation='relu')(f1)
    class2 = Dense(1 , activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048 , activation='relu')(f2)
    regress2 = Dense(4 , activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer , outputs=[class2 , regress2])
    return facetracker

In [64]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [66]:
facetracker = build_model()

In [68]:
facetracker.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, None, None,   14714688    ['input_6[0][0]']                
                                512)                                                              
                                                                                                  
 global_max_pooling2d_4 (Global  (None, 512)         0           ['vgg16[0][0]']                  
 MaxPooling2D)                                                                              